In [1]:
!pip install gekko
!pip install panel
from gekko import GEKKO 
import pandas as pd
import re
import panel as pn
import sys
import time
import ipywidgets as widgets
from IPython.display import display
pn.extension()
#sys.tracebacklimit=0

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


# Functions

In [2]:
# Get user input values
def get_inputs(event):
  global fresh_density
  global fresh_density_flag                                     # Defined - 1 not defined - 0
  global bulk_density
  global bulk_density_flag
  #global water_demand
  #global water_demand_flag
  global yield_liter_per_ton
  global yield_liter_per_ton_flag
  global flex_strength
  global flex_strength_flag
  global compr_strength
  global compr_strength_flag
  global dry_density
  global dry_density_flag
  global types_exist

  if(text_input_fresh.value_input != ''):
    fresh_density = float(text_input_fresh.value_input)
    fresh_density_flag = 1
  else:
    try:
      fresh_density
    except NameError:
      fresh_density_flag = 0
    else:
      fresh_density_flag = 0
      del fresh_density

  if(text_input_bulk.value_input != ''):
    bulk_density = float(text_input_bulk.value_input)
    bulk_density_flag = 1
  else:
    try:
      bulk_density
    except NameError:
      bulk_density_flag = 0
    else:
      bulk_density_flag = 0
      del bulk_density

  #if(text_input_water.value_input != ''):
    #water_demand = float(text_input_water.value_input)
    #water_demand_flag = 1
  #else:
    #try:
      #water_demand
    #except NameError:
      #water_demand_flag = 0
    #else:
      #water_demand_flag = 0
      #del water_demand

  if(text_input_yield.value_input != ''):
    yield_liter_per_ton = float(text_input_yield.value_input)
    yield_liter_per_ton_flag = 1
  else:
    try:
      yield_liter_per_ton
    except NameError:
      yield_liter_per_ton_flag = 0
    else:
      yield_liter_per_ton_flag = 0
      del yield_liter_per_ton

  if(text_input_flex.value_input != ''):
    flex_strength = float(text_input_flex.value_input)
    flex_strength_flag = 1
  else:
    try:
      flex_strength
    except NameError:
      flex_strength_flag = 0
    else:
      flex_strength_flag = 0
      del flex_strength

  if(text_input_compr.value_input != ''):
    compr_strength = float(text_input_compr.value_input)
    compr_strength_flag = 1
  else:
    try:
      compr_strength
    except NameError:
      compr_strength_flag = 0
    else:
      compr_strength_flag = 0
      del compr_strength

  if(text_input_dry.value_input != ''):
    dry_density = float(text_input_dry.value_input)
    dry_density_flag = 1
  else:
    try:
      dry_density
    except NameError:
      dry_density_flag = 0
    else:
      dry_density_flag = 0
      del dry_density
  
  # Show ui one time only  
  #if(types_exist != 1):
    #types_exist=1
    #make_radio()

  select_types()

def create_model():
  # Creation of GEKKO model
  global m
  m = GEKKO()                     

  m.options.SOLVER=2

def load_variables():
  global cement_dos
  global vinna_dos
  global abm
  global aero_dos
  global water_demand

  abm = abm_val                                       # Constant
 
  cement_dos = m.Var(lb=cd_lb, ub=cd_ub)              # lb = lower bound, ub = upperbound
  vinna_dos = m.Var(lb=vd_lb, ub=vd_ub)
  aero_dos = m.Var(lb=ad_lb, ub=ad_ub)
  water_demand = m.Var()

  if(fresh_density_flag == 0):
    global fresh_density
    fresh_density = m.Var()
  if(bulk_density_flag == 0):
    global bulk_density
    bulk_density = m.Var()
  #if(water_demand_flag == 0):
    #global water_demand
    #water_demand = m.Var()
  if(yield_liter_per_ton_flag == 0):
    global yield_liter_per_ton
    yield_liter_per_ton = m.Var()
  if(flex_strength_flag == 0):
    global flex_strength
    flex_strength = m.Var()
  if(compr_strength_flag == 0):
    global compr_strength
    compr_strength = m.Var()
  if(dry_density_flag == 0):
    global dry_density
    dry_density = m.Var()

# Create Equations
def load_equation(str_eq1, str_eq2, str_eq3, str_eq4, str_eq5, str_eq6, str_eq7):
  mat_eq1 = eval(str_eq1)
  mat_eq2 = eval(str_eq2)
  mat_eq3 = eval(str_eq3)
  mat_eq4 = eval(str_eq4)
  mat_eq5 = eval(str_eq5)
  mat_eq6 = eval(str_eq6)
  mat_eq7 = eval(str_eq7)
  
  m.Equations([mat_eq1 - fresh_density       == 0, #Eq1
               mat_eq2 - bulk_density        == 0, #Eq2
               mat_eq3 - water_demand        == 0, #Eq3
               mat_eq4 - yield_liter_per_ton == 0, #Eq4
               mat_eq5 - flex_strength       == 0, #Eq5
               mat_eq6 - compr_strength      == 0, #Eq6
               mat_eq7 - dry_density         == 0])#Eq7 
  return str_eq1, str_eq2, str_eq3, str_eq4, str_eq5, str_eq6, str_eq7

# Solving
def solve_equation():

  #only_solve_i = time.time()

  m.solve(disp=False)

  #only_solve_f = time.time()

  #print("Only solve time: ", only_solve_f - only_solve_i)
  # Solution Table

  result = []

  result.append(cement_dos.value[0])
  result.append(vinna_dos.value[0])
  result.append(aero_dos.value[0])

  if(fresh_density_flag == 0):
    #print("Fresh Density: " + str(fresh_density.value[0]))
    result.append(fresh_density.value[0])
  else:
    #print("Fresh Density: " + str(fresh_density))
    result.append(fresh_density)
    
  if(bulk_density_flag == 0):
    #print("Bulk Density: " + str(bulk_density.value[0]))
    result.append(bulk_density.value[0])
  else:
    #print("Bulk Density: " + str(bulk_density))
    result.append(bulk_density)

  #if(water_demand_flag == 0):
    #print("Water Demand: " + str(water_demand.value[0]))
    #result.append(water_demand.value[0])
  #else:
    #print("Water Demand: " + str(water_demand))
  result.append(water_demand.value[0])

  if(yield_liter_per_ton_flag == 0):
    #print("Yield ltr/ton: " + str(yield_liter_per_ton.value[0]))
    result.append(yield_liter_per_ton.value[0])
  else:
    #print("Yield ltr/ton: " + str(yield_liter_per_ton))
    result.append(yield_liter_per_ton)

  if(flex_strength_flag == 0):
    #print("Flex Strength: " + str(flex_strength.value[0]))
    result.append(flex_strength.value[0])
  else:
    #print("Flex Strength: " + str(flex_strength))
    result.append(flex_strength)

  if(compr_strength_flag == 0):
    #print("Compr Strength: " + str(compr_strength.value[0]))
    result.append(compr_strength.value[0])
  else:
    #print("Compr Strength: " + str(compr_strength))
    result.append(compr_strength)

  if(dry_density_flag == 0):
    #print("Dry Density: " + str(dry_density.value[0])  + "\n" )
    result.append(dry_density.value[0])
  else:
    #print("Dry Density: " + str(dry_density)  + "\n" )
    result.append(dry_density)

  all_results.append(result)

# Start program

def initialize(cd_lb_val, cd_ub_val, vd_lb_val, vd_ub_val, ad_lb_val, ad_ub_val, abm_const, max_size):
  global show_result_flag
  global cd_lb
  global cd_ub
  global vd_lb
  global vd_ub
  global ad_lb
  global ad_ub
  global abm_val
  global disp_size
    
  cd_lb = cd_lb_val
  cd_ub = cd_ub_val
  vd_lb = vd_lb_val  
  vd_ub = vd_ub_val
  ad_lb = ad_lb_val
  ad_ub = ad_ub_val
  abm_val = abm_const
  disp_size = max_size

  show_result_flag = False
  make_radio()


def build_model(fresh_eq, bulk_eq, water_eq, yield_eq, flex_eq, compr_eq, dry_eq):
  # Creation of Gekko model
  create_model()

  # Variables
  load_variables()

  # Equations  
  load_equation(fresh_eq, bulk_eq, water_eq, yield_eq, flex_eq, compr_eq, dry_eq)

# Main

def main(bulk_eq, yield_eq, fresh_eq, water_eq, flex_eq, compr_eq, dry_eq):

  # Character Correction
  
  fresh_eq = fresh_eq.replace("–", "-") # Left one gives error
  water_eq = water_eq.replace("–", "-")
  bulk_eq = bulk_eq.replace("–", "-")
  yield_eq = yield_eq.replace("–", "-")
  flex_eq = flex_eq.replace("–", "-")
  compr_eq = compr_eq.replace("–", "-")
  dry_eq = dry_eq.replace("–", "-")

  #water_eq = water_eq.replace(",",".")  # 3,5 to 3.5
  #water_eq = water_eq.replace("^","**")
  
  # Extract digit from string
  #bulk_str = re.findall(r'\-?\s?\d+', bulk_eq)
  #yield_str = re.findall(r'\-?\s?\d+', yield_eq)

  # Creation of Gekko model
  create_model()

  # Variables
  load_variables()

  # Equations  
  str_eq1, str_eq2, str_eq3, str_eq4, str_eq5, str_eq6, str_eq7 = load_equation(fresh_eq, bulk_eq, water_eq, yield_eq, flex_eq, compr_eq, dry_eq)

  # Solve
  global all_results
  global sol_time
  show_result_flag = True
  all_results = []
  stop_flag = False

  # Find increase size
    
  inc_size = 1/(((disp_size/(((vd_ub-vd_lb)/0.25)+1))-1)/(cd_ub-cd_lb))
  total_inc_size = disp_size/(((vd_ub-vd_lb)/0.25)+1)
  per_percent_inc = 100/(total_inc_size-1)
  percent = 0
  iteration_num = 1

  initial_time = time.time()
  lb_cement_dos = cd_lb
  while lb_cement_dos <=cd_ub:
    print("Working ... : " + str(round(percent,2)) + "%")
    cement_dos.LOWER = lb_cement_dos
    lb_vinna_dos = vd_lb
    while lb_vinna_dos <= vd_ub:
      vinna_dos.LOWER = lb_vinna_dos
      try:
        solve_equation()
      except Exception:
        if(lb_cement_dos == cd_lb and lb_vinna_dos == vd_lb):
          stop_flag = True
          break
      lb_vinna_dos = lb_vinna_dos + 0.25
    if stop_flag == True:
      break
    percent = percent + per_percent_inc
    if iteration_num == total_inc_size-1 :  
      lb_cement_dos = round(lb_cement_dos + inc_size)
    else:
      lb_cement_dos = lb_cement_dos + inc_size

    iteration_num = iteration_num + 1
    m.cleanup()
    build_model(fresh_eq, bulk_eq, water_eq, yield_eq, flex_eq, compr_eq, dry_eq)
        
  final_time = time.time()
  sol_time = final_time-initial_time

  if show_result_flag == True:
    cost_func(str_eq1, str_eq2, str_eq3, str_eq4, str_eq5, str_eq6, str_eq7)

def cost_func(str_eq1, str_eq2, str_eq3, str_eq4, str_eq5, str_eq6, str_eq7):
      
      df = show_results()
      
      str_eq1 = str_eq1 + "-"
      str_eq2 = str_eq2 + "-"
      str_eq3 = str_eq3 + "-"
      str_eq4 = str_eq4 + "-"
      str_eq5 = str_eq5 + "-"
      str_eq6 = str_eq6 + "-"
      str_eq7 = str_eq7 + "-"
      
      cost = 0
      # Accuracy test
      for i in range(df.shape[0]):
        cement_dos = df['cement_dos'][i]
        vinna_dos = df['vinna_dos'][i]
        aero_dos = df['aero_dos'][i]
        fresh_density = df['fresh_density'][i]
        bulk_density = df['bulk_density'][i]
        water_demand = df['water_demand'][i]
        yield_liter_per_ton = df['yield_liter_per_ton'][i]
        flex_strength = df['flex_strength'][i]
        compr_strength = df['compr_strength'][i]
        dry_density = df['dry_density'][i]

        acc_1 = eval(str_eq1 + "fresh_density")
        acc_2 =eval(str_eq2 + "bulk_density")
        acc_3 =eval(str_eq3 + "water_demand")
        acc_4 =eval(str_eq4 + "yield_liter_per_ton")
        acc_5 =eval(str_eq5 + "flex_strength")
        acc_6 =eval(str_eq6 + "compr_strength")
        acc_7 =eval(str_eq7 + "dry_density")

        row_acc = (acc_1 + acc_2 + acc_3 + acc_4 + acc_5 + acc_6 + acc_7)/7
        cost = cost + row_acc
      if str(cost) !='0' :
        cost = cost / df.shape[0]
        print("Cost: " + str(cost))
  
  # Show output in table
def show_results():
  df = pd.DataFrame(columns=['cement_dos', 'vinna_dos', 'aero_dos', 'fresh_density', 'bulk_density', 'water_demand',
      'yield_liter_per_ton', 'flex_strength', 'compr_strength', 'dry_density'], data=all_results)
  table_title = cement_val + " " +  aero_val + " " + sand_val + " "
  df.drop_duplicates(keep="first",inplace=True)
  df.sort_values('cement_dos', inplace =True)
  df.reset_index(inplace=True, drop = True)
  df = df.style.set_caption(table_title)

  if len(df.index) == 0:
      print("Solution not found!")
  else:
      display(df)
      print("Solution time " + str(sol_time) + " seconds ")	
  return df.data
      

def make_text_box():
  global text_input_fresh
  global text_input_bulk
  global text_input_water
  global text_input_yield
  global text_input_flex
  global text_input_compr
  global text_input_dry

  text_input_fresh = pn.widgets.TextInput(name='Fresh Density: ', width=100, placeholder='')
  text_input_bulk = pn.widgets.TextInput(name='Bulk Density: ', width=100, placeholder='')
  #text_input_water = pn.widgets.TextInput(name='Water Demand: ', width=100, placeholder='')
  text_input_yield = pn.widgets.TextInput(name='Yield liter/ton: ', width=100, placeholder='')
  text_input_flex = pn.widgets.TextInput(name='Flex Strength: ', width=100, placeholder='')
  text_input_compr = pn.widgets.TextInput(name='Compr Strength: ', width=100, placeholder='')
  text_input_dry = pn.widgets.TextInput(name='Dry Density: ', width=100, placeholder='')

  row1 = pn.Row(text_input_fresh, text_input_bulk, text_input_yield)
  row2 = pn.Row(text_input_flex, text_input_compr, text_input_dry)

  display(row1)
  display(row2)

  #enter_button = pn.widgets.Button(name = 'Enter', button_type = 'primary', width=220)
  enter_button = widgets.Button(description="Solve", button_style = 'success')
  enter_button.layout.width = '220px'
  enter_button.layout.margin = '0 0 0 10px'
  display(enter_button)
  enter_button.on_click(get_inputs)

def make_radio():
  
  global radio_group_cement
  global radio_group_aero
  global radio_group_sand
  
  radio_group_cement = pn.widgets.RadioButtonGroup(
      name='cement_type', options=['Dyckerhoff', 'Milke'], button_type='default', width=220)
  display(radio_group_cement)

  radio_group_aero = pn.widgets.RadioButtonGroup(
      name='aeropor_type', options=['Aeropor 500', 'Aeropor 1000'], button_type='default', width=220)
      
  display(radio_group_aero)

  radio_group_sand = pn.widgets.RadioButtonGroup(
      name='sand_type', options=['Limestone', 'Quartz'], button_type='default', width=220)
  display(radio_group_sand)

  #select_button = pn.widgets.Button(name = 'Select', button_type = 'success', width=220)

  #display(select_button)

  make_text_box()

  #select_button.on_click(select_types)

def select_types():

  global cement_val
  global aero_val
  global sand_val

  cement_val = radio_group_cement.value
  aero_val = radio_group_aero.value
  sand_val = radio_group_sand.value

  if(cement_val == "Dyckerhoff" and aero_val == "Aeropor 500" and sand_val == "Limestone"):
    dyc_500_lime()
  elif(cement_val == "Dyckerhoff" and aero_val == "Aeropor 500" and sand_val == "Quartz"):
    dyc_500_quar()
  elif(cement_val == "Dyckerhoff" and aero_val == "Aeropor 1000" and sand_val == "Limestone"):
    dyc_1000_lime()
  elif(cement_val == "Dyckerhoff" and aero_val == "Aeropor 1000" and sand_val == "Quartz"):
    dyc_1000_quar()
  elif(cement_val == "Milke" and aero_val == "Aeropor 500" and sand_val == "Limestone"):
    mil_500_lime()
  elif(cement_val == "Milke" and aero_val == "Aeropor 500" and sand_val == "Quartz"):
    mil_500_quar()
  elif(cement_val == "Milke" and aero_val == "Aeropor 1000" and sand_val == "Limestone"):
    mil_1000_lime()
  elif(cement_val == "Milke" and aero_val == "Aeropor 1000" and sand_val == "Quartz"):
    mil_1000_quar()

# Formules

In [3]:
# Dyckerhoff A_500 Limestone
def dyc_500_lime():

  #print("Selected: Dyckerhoff A_500 Limestone")

  fresh_eq = "1308 -25*aero_dos"
  bulk_eq = "1217 -22*aero_dos"
  water_eq = "2.92 -0.58*aero_dos +1.1*abm"
  yield_eq = "725 +43.4*aero_dos"
  flex_eq = "-0.72 +0.091*cement_dos -0.073*aero_dos +0.22*vinna_dos"
  compr_eq = "-0.67 +0.2*cement_dos -0.13*aero_dos"
  dry_eq = "1034 +6.9*cement_dos -24.93*aero_dos"

  main(bulk_eq, yield_eq, fresh_eq, water_eq, flex_eq, compr_eq, dry_eq)

# Dyckerhoff A_500 Quartz
def dyc_500_quar():

  #print("Selected: Dyckerhoff A_500 Quartz")

  fresh_eq = "1308 -25*aero_dos"
  bulk_eq = "1238 – 21.7*aero_dos"
  water_eq = "2.92 -0.58*aero_dos +1.1*abm"
  yield_eq = "725 +43.4*aero_dos"
  flex_eq = "-0.72 +0.091*cement_dos -0.073*aero_dos +0.22*vinna_dos"
  compr_eq = "-0.67 +0.2*cement_dos -0.13*aero_dos"
  dry_eq = "1034 +6.9*cement_dos -24.93*aero_dos"


  main(bulk_eq, yield_eq, fresh_eq, water_eq, flex_eq, compr_eq, dry_eq)

# Dyckerhoff A_1000 Limestone
def dyc_1000_lime():

  #print("Selected: Dyckerhoff A_1000 Limestone")

  fresh_eq = "1287 -25*aero_dos"
  bulk_eq = "1238 - 27*aero_dos"
  water_eq = "2.92 -0.58*aero_dos +1.1*abm" 
  yield_eq = "779 +43.4*aero_dos"
  flex_eq = "-0.72 +0.091*cement_dos -0.073*aero_dos +0.22*vinna_dos"
  compr_eq = "-0.67 +0.2*cement_dos -0.13*aero_dos"
  dry_eq = "1012 +6.9*cement_dos -24.93*aero_dos"

  main(bulk_eq, yield_eq, fresh_eq, water_eq, flex_eq, compr_eq, dry_eq)

# Dyckerhoff A_1000 Quartz
def dyc_1000_quar():

  #print("Selected: Dyckerhoff A_1000 Quartz")

  fresh_eq = "1287 -25*aero_dos" 
  bulk_eq = "1260 - 27*aero_dos"
  water_eq = "2.92 -0.58*aero_dos +1.1*abm" 
  yield_eq = "779 +43.4*aero_dos"
  flex_eq = "-0.72 +0.091*cement_dos -0.073*aero_dos +0.22*vinna_dos"
  compr_eq = "-0.67 +0.2*cement_dos -0.13*aero_dos"
  dry_eq = "1012 +6.9*cement_dos -24.93*aero_dos"


  main(bulk_eq, yield_eq, fresh_eq, water_eq, flex_eq, compr_eq, dry_eq)  

# # Milke A_500 Limestone
def mil_500_lime():

  #print("Selected: Milke A_500 Limestone")

  fresh_eq = "1308 -25*aero_dos"
  bulk_eq = "1217 - 22*aero_dos"
  water_eq = "2.92 -0.58*aero_dos +1.1*abm" 
  yield_eq = "725 +43.4*aero_dos"
  flex_eq = "-0.34 +0.091*cement_dos -0.073*aero_dos +0.22*vinna_dos"
  compr_eq = "0.29 +0.2*cement_dos -0.13*aero_dos"
  dry_eq = "1034 +6.9*cement_dos -24.93*aero_dos"

  main(bulk_eq, yield_eq, fresh_eq, water_eq, flex_eq, compr_eq, dry_eq)

# Milke A_500 Quartz
def mil_500_quar():

  #print("Selected: Milke A_500 Quartz")

  fresh_eq = "1308 -25*aero_dos"
  bulk_eq = "1238 - 22*aero_dos"
  water_eq = "2.92 -0.58*aero_dos +1.1*abm" 
  yield_eq = "725 +43.4*aero_dos"
  flex_eq = "-0.34 +0.091*cement_dos -0.073*aero_dos +0.22*vinna_dos"
  compr_eq = "0.29 +0.2*cement_dos -0.13*aero_dos"
  dry_eq = "1034 +6.9*cement_dos -24.93*aero_dos"

  main(bulk_eq, yield_eq, fresh_eq, water_eq, flex_eq, compr_eq, dry_eq)

# Milke A_1000 Limestone
def mil_1000_lime():

  #print("Selected: Milke A_1000 Limestone")

  fresh_eq = "1287 -25*aero_dos"
  bulk_eq = "1238 - 27*aero_dos"
  water_eq = "2.92 -0.58*aero_dos +1.1*abm" 
  yield_eq = "779 +43.4*aero_dos"
  flex_eq = "-0.34 +0.091*cement_dos -0.073*aero_dos +0.22*vinna_dos"
  compr_eq = "-0.19 +0.2*cement_dos -0.13*aero_dos"
  dry_eq = "1012 +6.9*cement_dos -24.93*aero_dos"



  main(bulk_eq, yield_eq, fresh_eq, water_eq, flex_eq, compr_eq, dry_eq)    

# Milke A_1000 Quartz
def mil_1000_quar():

  #print("Selected: Milke A_1000 Quartz")

  fresh_eq = "1287 -25*aero_dos" 
  bulk_eq = "1260 - 27*aero_dos"
  water_eq = "2.92 -0.58*aero_dos +1.1*abm"
  yield_eq = "779 +43.4*aero_dos"
  flex_eq = "-0.34 +0.091*cement_dos -0.073*aero_dos +0.22*vinna_dos"
  compr_eq = "-0.19 +0.2*cement_dos -0.13*aero_dos"
  dry_eq = "1012 +6.9*cement_dos -24.93*aero_dos"


  main(bulk_eq, yield_eq, fresh_eq, water_eq, flex_eq, compr_eq, dry_eq)

# Testing


In [5]:
cd_lb_val = 16                                        # cement lower bound
cd_ub_val = 24                                        # cement upper bound
vd_lb_val = 3                                         # vinna lower bound
vd_ub_val = 4                                         # vinna upper bound
ad_lb_val = 10                                        # aero lower bound
ad_ub_val = 20                                        # aero upper bound
abm_const = 14                                        # abm constant value
max_size = 100                                         # maximum data size to display

initialize(cd_lb_val, cd_ub_val, vd_lb_val, vd_ub_val, ad_lb_val, ad_ub_val, abm_const, max_size)


RadioButtonGroup(name='cement_type', options=['Dyckerhoff', 'Milke'], value='Dyckerhoff', width=220)

RadioButtonGroup(name='aeropor_type', options=['Aeropor 500', ...], value='Aeropor 500', width=220)

RadioButtonGroup(name='sand_type', options=['Limestone', 'Quartz'], value='Limestone', width=220)

Row
    [0] TextInput(name='Fresh Density: ', width=100)
    [1] TextInput(name='Bulk Density: ', width=100)
    [2] TextInput(name='Yield liter/ton: ', width=100)

Row
    [0] TextInput(name='Flex Strength: ', width=100)
    [1] TextInput(name='Compr Strength: ', width=100)
    [2] TextInput(name='Dry Density: ', width=100)

Button(button_style='success', description='Solve', layout=Layout(margin='0 0 0 10px', width='220px'), style=B…